# CSCI 3155 Spring 2023

## Recitation Week 9

We have been learning and working with the [**Lettuce**](https://github.com/sriram0339/csci3155_notebooks/blob/master/5/Lettuce%20-%20The%20Let%20Language.ipynb)(**NB9**) over the last many weeks. 
It started as a language with *variables*, *arithmetic*, but now has grown to include
* *static scoping* : [Scopes and Environments](https://github.com/sriram0339/csci3155_notebooks/blob/master/5/Lettuce-Scopes-Environments.ipynb)(**NB10**)
* *functions* : [Function calls](https://github.com/sriram0339/csci3155_notebooks/blob/master/6/Lettuce-Function-Calls.ipynb)(**NB11**)
* *recursion* : [Recursion](https://github.com/sriram0339/csci3155_notebooks/blob/master/7/RecursionInLettuce.ipynb)(**NB12**)
* *references* : [References](https://github.com/sriram0339/csci3155_notebooks/blob/master/8/References-And-Mutable-Vars-Lettuce.ipynb)(**NB13**)

## Interpreting Lettuce

In this recitation we will focus on being able to interpret **Lettuce** programs incrementally building with features.

### Environment

We have defined an environment as a function from identifiers to values denoted by them. What sort of environments have we encountered?

* The empty environment: implemented as an empty map. Let us call this environment: $EmptyEnv$
* The environment $\sigma[x \mapsto v]$ which denotes a previously existing environment $\sigma$ extended with the mapping $x \mapsto v$ that associates identifier $x$ with value $v$. Let us call this operation $\texttt{Extend}(\sigma, x, v)$.

# #1 Example
## What is the result executing the program? 

~~~
let addOne = function(x) { x + 1 } in
    addOne(addOne(1))
~~~

## Let's execute it *step-by-step* and write down the environment (at each step) to understand it better


### Things to know:
* $ Closure(\mathbf{Identifier}, \mathbf{Expr}, \mathbf{Environment})$ or $\sigma(x, e, \hat{\sigma})$ is used to support static scoping by storing the definition environment - (NB11, rule *fun-def-ok*)
* Closure is also a **Value** like $ NumValue(\mathbf{Number})$ and can be mapped in an environment
* Function Call has two parts, the function and argument - (NB11, rule *fun-call-ok*)

Useful notations for writing:
* We can use $ \{\} $ for an empty environment

* We can use $ \{x -> NumValue(3.5)\} $ for an environment with "x" mapped to the numerical value 3.5

* Writing can be a flexibile medium, so we can use arrows (how the professor does in lecture) when refering to a version of the environment

### Note: Solutions
The solutions are written with a mix of code and text, and many steps are combined into a single summary to make the content concise yet complete.

### Solution: 

### Final value : 3

#### Step 1&2 
Ref: (NB9, *let-binding-ok*), (NB11, *fun-def-ok*)
~~~
let addOne = function(x) { x + 1 } in
    ...
~~~

We have a $Let(...)$ binding with a function definition as the value $Expr$. We know from the semantics (NB11, rule *fun-def-ok*) that a function definition is evaluated to a be $Closure$.

Combining this with (NB9, rule *let-binding-ok*) we need to extend the environment to map the $\mathbf{Identifier}$(addOne) to value $Closure("x", Plus(Ident("x"), NumValue(1.0), EmptyEnv)$ leading to the new environment $\sigma_1 = Extend("addOne", Closure("x", Plus(Ident("x"), NumValue(1.0), EmptyEnv), EmptyEnv)$

#### Step 3
Ref: (NB9, *let-binding-ok*), (NB11, *fun-call-ok*)
Continuing with the semantics (NB9, *let-binding-ok*) we need to evaluate the body with this new environment $\sigma_1$
~~~
addOne(addOne(1))
~~~

The body is a function call, it has two parts the `func_expr` and `arg_expr`.

`fun_expr` has to be a  closure, but it is $Ident("addOne")$ we look it up in the environment $\sigma_1$ and it is the $Closure$ that we mapped in the environment, which is ok according to the semantics.

`arg_expr` is another function call. We need to evaluate it with $\sigma_1$ before we can move forward. (NB11, *fun-call-ok*)

#### Step 4
~~~
...(addOne(1))
~~~

This is also function call, so we need to repeat things we did in the last step (NB11, *fun-call-ok*), `func_expr` is a closure, so we move on to  `arg_expr`, it is a value already, $NumValue(1.0)$ so we can go ahead and evaluate the closure expression that is the body of the function, with the value mapped to the argument.

$ \sigma_2 = Extend("x", 1, \sigma_1) $

#### Step 5
We have an arithmetic expression (NB9, *arith-binop-ok-rule*), with left operand being an $Ident(...)$, we can look it up in the environment $\sigma_2$ and get the value, it is 1.0, we can move ahead and do the math, we get 2.0

#### Step 6
We can resume from Step 3
~~~
addOne(addOne(1))
~~~

but we know `arg_expr` is 2.0 and `func_expr` is a $Closure$. 
We can now map the parameter to the value, with parameter $x$ being $2.0$, keep in mind the environment at this point is $\sigma_1$ and not $sigma_2$, we can extend it and get the new enviroment

$ \sigma_3 = Extend("x", 2.0, \sigma_1) $


#### Step 7
It is same as what we did in Step 5 just that x is different in $\sigma_3$, so our result will be 3.0

---

# #2 Exercise

## Execute the below program *step-by-step* and write down the environment (at each step)

~~~
let z = 50 in
let rec foo = function (x) {
            if (x <= 0){
                0
            } else {
                2 * foo(x-1) + z
            }
in
foo(3)
~~~

Note: We use 
* A third kind of extension of Environment is usedto support recursion:
    $\texttt{ExtendRec}(f, x, \texttt{e}, \sigma)$ which creates a new environment $\color{red}{\hat{\sigma}}$ such that $\color{red}{\hat{\sigma}}(x) = \sigma(x)$ for all identifiers $x \not= f$ and $\color{red}{\hat{\sigma}}(f) = \texttt{Closure}(x, \texttt{<body of function expr>}, \color{red}{\hat{\sigma}})$


### Solution

### Final value : 350

#### Step 1
~~~
let z = 50 in 
    ...
~~~

This program is (always?) made of a `let-binding`
We map "z" to the value $NumValue(50.0)$ in the environment, we get  have a new environment $\sigma_1$.

$\sigma_1 = Extend("z", NumValue(50.0), EmptyEnv)$

The body of this let a `let-rec`

#### Step 2 
~~~
let rec foo = function(x) {...}
    in 
    ...
~~~
The `let-rec` is similar from `let-binding` (NB12). The $Expr$ on the right is a function, it will be evaluated to a $Closure$ value. 

We need to map $foo$ to the function in the environment, but keep n mind that it is a `let-rec` and we need to use $ExtendRec$ to extend the environment. We will have a new environment.

$\sigma_2 = ExtendRec("foo", "x",  fBody, \sigma_1)$

#### Step 3

~~~
foo(3)
~~~
We need to evaluate the body of the $let-rec$ with new environment $\sigma_2$. The body expression is `foo(3)` which is a functional call.

We look up the $Ident("foo")$  in $\sigma_2$. Since it is a $ExtendRec$ made from a `let-rec` binding.

Supporting recursion looks us to support a circular scope, so a *special* closure is constructed during the lookup where the `closure_env` is additionally extended to include the function $foo$ itself ($\sigma_2$?).

It is a new closure $Closure("x", fBody, \sigma_2)$, where the `closure_env` is $\sigma_2$ instead of the $\sigma_1$.

#### Step 4 and so on
We will evaluate the function body, in base case it is a simple case of the base expression.

In recursive case, we look up `foo` in $\sigma_2$ again, repeating the process in Step #3. 

---

# #3 Exercise

## Execute the below program *step-by-step* and write down the *environment* and the *store* (at each step)

~~~
let n = 10 in
  let rec foo = function (x) 
                    function (y)
                    {
                         if (x <= 0) { NewRef(y) }
                         else NewRef( foo (x-1) (y) )
                    }
   in
    foo(n)(53)
~~~

### Things to know:
* Store : (NB12) Store like memory, it's a place with mutability, we can put things, update them, and get them using our language features $NewRef$, $AssignRef$ and $DeRef$ respectively.

### Useful notation
* $ Store $ can be written as 0-index array which is mutable (elements can be changed). A top-down layout may be very useful.


### Solution

### Final value : Reference(11)

### Store
| index | value |
| -- | -- |
|  0 |  NumValue(53.0) |
|  1 | Reference(0) |
|  2 | Reference(1) |
|  3 | Reference(2) |
|  4 | Reference(3) |
|  5 | Reference(4) |
|  6 | Reference(5) |
|  7 | Reference(6) |
|  8 | Reference(7) |
|  9 | Reference(8) |
|  10 | Reference(9) |
|  11 | Reference(10) |

### Store

#### Step #1

~~~
let n = 10 in 
    ...
~~~

We evaluate `let-binding` giving us a new environment $\sigma_1$

$\sigma_1 = Extend("n", NumValue(10.0), EmptyEnv)$

We move on to evaluate the `let-binding` body expression.

#### Step #2

~~~
    let rec foo = function(x) {...} in
        ...
~~~

This is a `let rec`, we repeat things we did in the previous problem step #2, we extend the environment with $ExtendRec$ leading to $\sigma_2$.

$\sigma_2 = ExtendRec("foo", "x", func\_expr, \sigma_1)$

We move to evaluating the body. 

#### Step #3
~~~
foo(n)(53)
~~~
We interpret the program left-to-right, so we need to evaluate just the `foo` function call. 

~~~
foo(n)
~~~

This is a function call, we know how to evaluate them. $foo$ is a $Ident("foo")$ we look it up the environment$\sigma_2$, we get a $Closure$ with closure environment being $\sigma_2$.

We will map the parameter to the argument $n$ which is $Ident("n")$ with value $NumValue(10.0)$ in closure environment ($\sigma_2$)

$\sigma_3 = Extend("x", NumValue(10.0), \sigma_2)$ 

We then move to evaluating the body with $\sigma_3$

#### Step #4

~~~
        {
            if (x <= 0) { NewRef(y) }
            else NewRef ( foo(x-1) (y) )
        }
~~~

The body of function `foo` is actually another function. This $FunDef(...)$ is evaluated into a closure with `fBody2`, and $\sigma_3$ and returned. 

#### Step #5
We come back to continuing to evalute the `let-binding` body.

~~~
Closure(...)(53)
~~~

This is again a function cal repeat the function-call steps, we extend the closure environment $\sigma_3$ with parameter mapped to $53$ and move to evaluate $fBody$.

$\sigma_4 = Extend("y", NumValue(53.0), \sigma_3)$ 

#### Step #6

The function has a `if-then-else` in both of which use of $NewRef$ which is a *side-effect* allocating a new cell on the store and storing the value. (NB13, rules **ite-then** and **ite-else**)

Since x is positive, we will evalute the else. 
The else contains `new` which is $NewRef(value\_expr)$ so we will make a new cell in $Store$ and store the value of  `value_expr` inside (pun intended). 

Note that `value_expr` is a function call on `foo`. We are still evaluating the top-level $foo$ call, so this is actually a recursion, although as a two-argument curried function.

#### Step #7 and so on
We are now evaluating a recursive function, although with store in tow. We know how to work with `new` so we can repeat what we normally do for recursion, with additional changes to store since both recursive and base case use store.

Note that, we are not using *AssignRef* which updates a *cell* in the $Store$ so we will have a series of values in the store, but keep in mind the order matters.


# #4 Exercise for after recitation
## Similar to #1
## Execute the below program *step-by-step* and write down the environment (at each step)

~~~
let x = 20 in
    let foo = function (z) {
        z - x 
        }
    in
    let bar = function (x) {
        2 * x
        }
    in 
    foo(20) + bar(10)

~~~

Solution will be posted on Piazza coming Tuesday. 

---